In [1]:
import pandas
pandas.options.mode.chained_assignment = None
import sys
import numpy as np
from globsML.utils.preprocessing import rescale_data, select_galaxies, create_data_dict, replace_NaN
from globsML.models.skmodels import StandardClassifier

data_path = '../../../data/ACS_sources_original.csv'
data = pandas.read_csv(data_path)
test_galaxies = set(data[data['cluster']=='Virgo']['galaxy'].unique())

In [2]:
df_train, dfs_test = select_galaxies(data, test_galaxies = test_galaxies)
df_train, dfs_test = replace_NaN(df_train, dfs_test)
df = create_data_dict(data, df_train, dfs_test)
df, _ = rescale_data(df)

input_dim = len(df['train']['inputs'][0])

Number of sources in training split after dropping rows with NaN as CI/m/color: 21232
535 sources have been dropped.
NaN values in testing data have been replaced with the correspnding median value observed in the training split


100%|█████████████████████████████████████████████████████████████████████████████████| 29/29 [00:00<00:00, 3802.28it/s]

eccentricity will not be transformed. Skipped.
eccentricity_z will not be transformed. Skipped.


In [3]:
model = StandardClassifier(method='catboost', data=df)

Loading data...
Fitting model...


In [4]:
model.val()

/home/dodo/anaconda3/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/dodo/anaconda3/lib/python3.8/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [5]:
model.stats_validation

,TPR,FDR,FPR,"AUC(FDR,TPR)","AUC(FPR,TPR)",# found GCs,# total GCs,# fake GCs,# sources
0,0.916933,0.085987,0.036048,0.947733,0.985736,287,313,27,1062


In [6]:
model.test()

/home/dodo/anaconda3/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/dodo/anaconda3/lib/python3.8/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/dodo/git/globsML/globsML/utils/eval.py:146: RuntimeWarning: divide by zero encountered in double_scalars
  return auc(x,y)/(2*auc([0,x[-1]], [0,y[-1]])), x, y


In [7]:
model.stats_galaxies

,Galaxy,TPR,FDR,FPR,"AUC(FDR,TPR)","AUC(FPR,TPR)",# found GCs,# total GCs,# fake GCs,# sources
0,VCC2092,0.898876,0.069767,0.020270,0.950767,0.977357,80,89,6,385
1,VCC1886,0.615385,0.428571,0.008043,0.694746,0.950557,8,13,6,759
2,VCC543,0.884615,0.361111,0.020833,0.816010,0.971493,23,26,13,650
3,VCC1407,0.864407,0.089286,0.006793,0.953773,0.995279,51,59,5,795
4,VCC1087,0.910448,0.115942,0.017505,0.947524,0.983997,61,67,8,524
...,...,...,...,...,...,...,...,...,...,...
93,VCC784,0.870968,0.068966,0.010025,0.973967,0.996382,54,62,4,461
94,VCC1242,0.876106,0.108108,0.029703,0.952723,0.986605,99,113,12,517
95,VCC1250,0.925926,0.152542,0.024523,0.955113,0.994096,50,54,9,421
96,VCC575,0.739130,0.190476,0.010667,0.877924,0.985449,17,23,4,398


In [8]:
model.stats_galaxies.mean()

/tmp/ipykernel_1992/3415850949.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  model.stats_galaxies.mean()


TPR               0.852567
FDR               0.171326
FPR               0.020176
AUC(FDR,TPR)           inf
AUC(FPR,TPR)      0.983824
# found GCs     113.969388
# total GCs     126.479592
# fake GCs        9.265306
# sources       644.510204
dtype: float64

In [9]:
model.stats_all

,Galaxy,TPR,FDR,FPR,"AUC(FDR,TPR)","AUC(FPR,TPR)",# found GCs,# total GCs,# fake GCs,# sources
0,ALL,0.901089,0.075184,0.017886,0.962242,0.989902,11169,12395,908,63162
